In [1]:
# Cell 1: Enhanced imports and setup
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.feature_selection import VarianceThreshold
from lightgbm import LGBMClassifier
import warnings
warnings.filterwarnings('ignore')

# For holiday calendar
try:
    from pandas.tseries.holiday import USFederalHolidayCalendar
    HOLIDAY_CALENDAR_AVAILABLE = True
except ImportError:
    HOLIDAY_CALENDAR_AVAILABLE = False
    print("Warning: Holiday calendar not available, will skip holiday features")

# Load the datasets
train_df = pd.read_csv('/kaggle/input/hopeless/train_kanafren.csv')
test_df = pd.read_csv('/kaggle/input/hopeless/test_kanafren.csv')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"Train columns: {len(train_df.columns)}")
print(f"Test columns: {len(test_df.columns)}")


Train shape: (770164, 391)
Test shape: (369301, 390)
Train columns: 391
Test columns: 390


In [2]:
# Cell 2: Strategic pre-imputation
def strategic_pre_imputation(df):
    """Strategic pre-imputation for features critical to feature engineering"""
    df_imputed = df.copy()
    
    # 1. Interest scores (f1-f12) - median by user activity level
    interest_cols = [f'f{i}' for i in range(1, 13) if f'f{i}' in df.columns]
    
    # Create user activity tier for grouping
    if 'id2' in df.columns:
        user_impression_counts = df.groupby('id2').size()
        df_imputed['user_activity_tier'] = pd.cut(
            df_imputed['id2'].map(user_impression_counts),
            bins=[0, 1, 5, 20, float('inf')],
            labels=['low', 'medium', 'high', 'very_high']
        )
        
        # Median imputation by activity tier
        for col in interest_cols:
            df_imputed[col] = df_imputed.groupby('user_activity_tier')[col].transform(
                lambda x: x.fillna(x.median())
            )
    else:
        # Fallback to global median
        for col in interest_cols:
            df_imputed[col] = df_imputed[col].fillna(df_imputed[col].median())
    
    # 2. Loyalty features - 0 for numerical (no participation), mode for categorical
    loyalty_numerical = ['f43', 'f46', 'f47', 'f51', 'f58'] # Miles, tenure, etc.
    loyalty_categorical = ['f42', 'f54', 'f55'] # Membership level, status
    
    for col in loyalty_numerical:
        if col in df_imputed.columns:
            df_imputed[col] = df_imputed[col].fillna(0)
    
    for col in loyalty_categorical:
        if col in df_imputed.columns:
            mode_val = df_imputed[col].mode().iloc[0] if not df_imputed[col].mode().empty else 'Unknown'
            df_imputed[col] = df_imputed[col].fillna(mode_val)
    
    # 3. Spending patterns (f152-f198) - 0 (no spending)
    spending_cols = [f'f{i}' for i in range(152, 199) if f'f{i}' in df.columns]
    for col in spending_cols:
        df_imputed[col] = df_imputed[col].fillna(0)
    
    # 4. CTR features (f104-f113, f310-f314) - global median for established users, 0 for new
    ctr_cols = [f'f{i}' for i in range(104, 114)] + [f'f{i}' for i in range(310, 315)]
    ctr_cols = [col for col in ctr_cols if col in df_imputed.columns]
    
    for col in ctr_cols:
        global_median = df_imputed[col].median()
        df_imputed[col] = df_imputed[col].fillna(global_median if global_median > 0 else 0)
    
    # 5. Transaction frequency features (f174-f198) - 0 (no transactions)
    transaction_cols = [f'f{i}' for i in range(174, 199) if f'f{i}' in df.columns]
    for col in transaction_cols:
        df_imputed[col] = df_imputed[col].fillna(0)
    
    # Clean up temporary column
    if 'user_activity_tier' in df_imputed.columns:
        df_imputed = df_imputed.drop(columns=['user_activity_tier'])
    
    return df_imputed

# Apply strategic pre-imputation
print("Applying strategic pre-imputation...")
train_df = strategic_pre_imputation(train_df)
test_df = strategic_pre_imputation(test_df)
print(f"Pre-imputation completed. Train shape: {train_df.shape}, Test shape: {test_df.shape}")


Applying strategic pre-imputation...
Pre-imputation completed. Train shape: (770164, 391), Test shape: (369301, 390)


In [3]:
# Rename columns to ensure consistency between train and test
if 'f374_x' in train_df.columns:
    train_df.rename(columns={'f374_x': 'f374'}, inplace=True)
if 'f374_x' in test_df.columns:
    test_df.rename(columns={'f374_x': 'f374'}, inplace=True)

# Check available offer metadata columns
offer_metadata_cols = ['id3', 'f374', 'f376', 'id8', 'id12', 'id13']
existing_cols = [col for col in offer_metadata_cols if col in train_df.columns]
print("Available offer metadata columns:", existing_cols)

# Display sample data
print("\nSample offer metadata (train):")
print(train_df[existing_cols].head())

Available offer metadata columns: ['id3', 'f374', 'f376', 'id8', 'id12', 'id13']

Sample offer metadata (train):
         id3                                               f374  f376  \
0  189706075  Radio, Television, And Consumer Electronics St...   2.0   
1      89227                                      Liquor Stores   NaN   
2      35046                                       Beauty Shops  10.0   
3    6275451                             Family Clothing Stores  10.0   
4      78053                   Miscellaneous Retail Stores, Nec   8.0   

          id8        id12                 id13  
0  57310000.0  2023-11-01  2023-11-30 23:59:59  
1  59210000.0  2023-11-01  2024-04-30 23:59:59  
2  72310000.0  2023-11-01  2023-11-30 23:59:59  
3  56510500.0  2023-11-01  2023-11-30 23:59:59  
4  59991300.0  2023-11-01  2023-11-30 23:59:59  


In [4]:
# Cell 3: Prepare datetime columns and sort data (CRITICAL FIX)
def prepare_temporal_data(df):
    """Prepare temporal data and sort by user and time to prevent data leakage"""
    df_prepared = df.copy()
    
    # Convert datetime columns
    df_prepared['impression_time'] = pd.to_datetime(df_prepared['id4'])
    df_prepared['click_time'] = pd.to_datetime(df_prepared['id7'], errors='coerce')
    
    # CRITICAL: Sort by user and time to enable proper temporal aggregations
    df_prepared = df_prepared.sort_values(['id2', 'impression_time']).reset_index(drop=True)
    
    # Create click indicator early
    df_prepared['has_clicked'] = df_prepared['click_time'].notna().astype(int)
    
    return df_prepared

# Apply temporal preparation
print("Preparing temporal data and sorting...")
train_df = prepare_temporal_data(train_df)
test_df = prepare_temporal_data(test_df)
print(f"Temporal preparation completed. Train shape: {train_df.shape}, Test shape: {test_df.shape}")


Preparing temporal data and sorting...
Temporal preparation completed. Train shape: (770164, 394), Test shape: (369301, 393)


In [5]:
# Cell 4: Data Analysis and Dynamic Threshold Calculation
def calculate_optimal_time_threshold(df, method='quantile', train_ratio=0.8, min_val_samples=1000):
    """Calculate optimal time threshold for train/validation split"""
    
    if 'impression_time' not in df.columns:
        raise ValueError("impression_time column not found")
    
    total_samples = len(df)
    
    print("Data time range analysis:")
    print(f"Earliest impression: {df['impression_time'].min()}")
    print(f"Latest impression: {df['impression_time'].max()}")
    print(f"Total date span: {(df['impression_time'].max() - df['impression_time'].min()).days} days")
    
    # Check data distribution by date
    daily_counts = df.groupby(df['impression_time'].dt.date).size()
    print(f"\nDaily data distribution:")
    print(daily_counts)
    
    if method == 'quantile':
        # Use quantile-based approach
        threshold = df['impression_time'].quantile(train_ratio)
        
    elif method == 'sample_based':
        # Ensure minimum validation samples
        sorted_dates = df['impression_time'].sort_values()
        val_samples_needed = max(min_val_samples, int(total_samples * (1 - train_ratio)))
        split_index = total_samples - val_samples_needed
        threshold = sorted_dates.iloc[split_index]
        
    elif method == 'daily_boundary':
        # Split on day boundaries to avoid splitting user sessions
        daily_counts_cumsum = daily_counts.cumsum()
        target_train_samples = int(total_samples * train_ratio)
        
        # Find the day where cumulative count exceeds target
        split_day = daily_counts_cumsum[daily_counts_cumsum >= target_train_samples].index[0]
        threshold = pd.Timestamp(split_day)
        
    else:
        raise ValueError(f"Unknown method: {method}")
    
    # Validate the threshold
    train_count = (df['impression_time'] < threshold).sum()
    val_count = (df['impression_time'] >= threshold).sum()
    
    print(f"\nThreshold calculation results:")
    print(f" Method: {method}")
    print(f" Threshold: {threshold}")
    print(f" Train samples: {train_count} ({train_count/total_samples:.1%})")
    print(f" Validation samples: {val_count} ({val_count/total_samples:.1%})")
    
    if val_count == 0:
        print("WARNING: No validation data with this threshold!")
        return None
        
    return threshold

# Calculate optimal threshold for your data
optimal_threshold = calculate_optimal_time_threshold(
    train_df, 
    method='quantile',  # Try 'sample_based' or 'daily_boundary' if needed
    train_ratio=0.8
)

print(f"\nOptimal threshold selected: {optimal_threshold}")


Data time range analysis:
Earliest impression: 2023-11-01 00:00:23.729000
Latest impression: 2023-11-03 23:59:58.572000
Total date span: 2 days

Daily data distribution:
impression_time
2023-11-01    301698
2023-11-02    246621
2023-11-03    221845
dtype: int64

Threshold calculation results:
 Method: quantile
 Threshold: 2023-11-03 08:30:56.586400
 Train samples: 616131 (80.0%)
 Validation samples: 154033 (20.0%)

Optimal threshold selected: 2023-11-03 08:30:56.586400


## Phase 1

In [6]:
# Cell 5: Merchant Category Features
def create_merchant_category_features(df):
    """Create merchant category features from f374 column"""
    df_features = df.copy()
    
    # One-hot encode merchant categories (limit to top categories to avoid high dimensionality)
    top_categories = df['f374'].value_counts().head(20).index
    df_features['f374_top'] = df_features['f374'].apply(lambda x: x if x in top_categories else 'Other')
    
    # Create dummy variables for top categories
    category_dummies = pd.get_dummies(df_features['f374_top'], prefix='merchant_cat')
    df_features = pd.concat([df_features, category_dummies], axis=1)
    
    # Category frequency encoding
    category_counts = df['f374'].value_counts()
    df_features['merchant_cat_frequency'] = df_features['f374'].map(category_counts)
    
    # Category rarity score (inverse frequency)
    df_features['merchant_cat_rarity'] = 1 / df_features['merchant_cat_frequency']
    
    return df_features

# Apply merchant category features
print("Creating merchant category features...")
train_df_features = create_merchant_category_features(train_df)
test_df_features = create_merchant_category_features(test_df)
print(f"Train shape after merchant features: {train_df_features.shape}")
print(f"Test shape after merchant features: {test_df_features.shape}")


Creating merchant category features...
Train shape after merchant features: (770164, 418)
Test shape after merchant features: (369301, 417)


In [7]:
# Cell 6: Enhanced Temporal Features
def create_enhanced_temporal_features(df):
    """Create enhanced temporal features with proper time-based logic"""
    df_features = df.copy()
    
    # Basic temporal components
    df_features['impression_hour'] = df_features['impression_time'].dt.hour
    df_features['impression_day_of_week'] = df_features['impression_time'].dt.dayofweek
    df_features['impression_day_of_month'] = df_features['impression_time'].dt.day
    df_features['impression_month'] = df_features['impression_time'].dt.month
    df_features['impression_quarter'] = df_features['impression_time'].dt.quarter
    
    # Time-based categories
    df_features['is_weekend_impression'] = (df_features['impression_day_of_week'] >= 5).astype(int)
    df_features['is_business_hours'] = (
        (df_features['impression_hour'] >= 9) & (df_features['impression_hour'] <= 17)
    ).astype(int)
    
    # Daypart interaction features
    df_features['weekend_business_hours'] = (
        df_features['is_weekend_impression'] & df_features['is_business_hours']
    ).astype(int)
    
    # Time of day categories
    df_features['time_of_day'] = pd.cut(
        df_features['impression_hour'],
        bins=[0, 6, 12, 18, 24],
        labels=['night', 'morning', 'afternoon', 'evening'],
        include_lowest=True
    )
    
    # Cyclical encoding for temporal features
    df_features['hour_sin'] = np.sin(2 * np.pi * df_features['impression_hour'] / 24)
    df_features['hour_cos'] = np.cos(2 * np.pi * df_features['impression_hour'] / 24)
    df_features['day_sin'] = np.sin(2 * np.pi * df_features['impression_day_of_week'] / 7)
    df_features['day_cos'] = np.cos(2 * np.pi * df_features['impression_day_of_week'] / 7)
    
    # Time since last interaction (FIXED for data leakage)
    df_features['time_since_last_impression'] = df_features.groupby('id2')['impression_time'].diff().dt.total_seconds()
    df_features['time_since_last_impression'] = df_features['time_since_last_impression'].fillna(0)
    
    # Time since last click (only for users who have clicked before)
    df_features['time_since_last_click'] = df_features.groupby('id2')['click_time'].diff().dt.total_seconds()
    df_features['time_since_last_click'] = df_features['time_since_last_click'].fillna(np.inf)
    
    # Holiday proximity features
    if HOLIDAY_CALENDAR_AVAILABLE:
        try:
            holiday_calendar = USFederalHolidayCalendar()
            holidays = holiday_calendar.holidays(
                start=df_features['impression_time'].min(), 
                end=df_features['impression_time'].max()
            )
            
            def days_to_nearest_holiday(date):
                if len(holidays) == 0:
                    return 365 # Default if no holidays
                return min([abs((date - h).days) for h in holidays])
            
            df_features['days_to_holiday'] = df_features['impression_time'].apply(days_to_nearest_holiday)
            df_features['is_near_holiday'] = (df_features['days_to_holiday'] <= 7).astype(int)
        except:
            df_features['days_to_holiday'] = 365
            df_features['is_near_holiday'] = 0
    else:
        df_features['days_to_holiday'] = 365
        df_features['is_near_holiday'] = 0
    
    return df_features

# Apply enhanced temporal features
print("Creating enhanced temporal features...")
train_df_features = create_enhanced_temporal_features(train_df_features)
test_df_features = create_enhanced_temporal_features(test_df_features)
print(f"Train shape after temporal features: {train_df_features.shape}")
print(f"Test shape after temporal features: {test_df_features.shape}")


Creating enhanced temporal features...
Train shape after temporal features: (770164, 435)
Test shape after temporal features: (369301, 434)


In [8]:
# Cell 7: Fixed User Session Features
def create_user_session_features_fixed(df):
    """Create user session features with proper temporal ordering to prevent data leakage"""
    df_features = df.copy()
    
    # CRITICAL: Data must be sorted by user and time before cumulative operations
    df_features = df_features.sort_values(['id2', 'impression_time']).reset_index(drop=True)
    
    # FIXED: Cumulative features (no future data leakage)
    df_features['user_cumulative_impressions'] = df_features.groupby('id2').cumcount()
    df_features['user_cumulative_clicks'] = df_features.groupby('id2')['has_clicked'].cumsum()
    
    # FIXED: Rolling CTR using proper approach for groupby + rolling
    df_features = df_features.set_index('impression_time')
    
    # Rolling CTR (7-day window, no future data)
    df_features['user_rolling_ctr_7d'] = (
        df_features.groupby('id2')['has_clicked']
        .rolling('7D', min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )
    
    # Rolling CTR (3-day window)
    df_features['user_rolling_ctr_3d'] = (
        df_features.groupby('id2')['has_clicked']
        .rolling('3D', min_periods=1)
        .mean()
        .reset_index(level=0, drop=True)
    )
    
    # Rolling click counts (no future data)
    df_features['clicks_last_3d'] = (
        df_features.groupby('id2')['has_clicked']
        .rolling('3D', min_periods=1)
        .sum()
        .reset_index(level=0, drop=True)
    )
    
    # Reset index to get impression_time back as a column
    df_features = df_features.reset_index()
    
    # User engagement velocity (change in CTR)
    df_features['user_ctr_trend'] = (
        df_features['user_rolling_ctr_3d'] - 
        df_features.groupby('id2')['user_rolling_ctr_3d'].shift(1)
    ).fillna(0)
    
    # Session sequence features
    df_features['offer_position_in_session'] = (
        df_features.groupby(['id2', df_features['impression_time'].dt.date]).cumcount() + 1
    )
    
    # User activity patterns
    df_features['user_session_length'] = df_features.groupby(['id2', df_features['impression_time'].dt.date])['id3'].transform('count')
    
    # User consistency metrics
    df_features['user_avg_session_length'] = (
        df_features.groupby('id2')['user_session_length']
        .expanding()
        .mean()
        .reset_index(level=0, drop=True)
    )
    
    return df_features

# Apply fixed user session features
print("Creating fixed user session features...")
train_df_features = create_user_session_features_fixed(train_df_features)
test_df_features = create_user_session_features_fixed(test_df_features)
print(f"Train shape after user session features: {train_df_features.shape}")
print(f"Test shape after user session features: {test_df_features.shape}")


Creating fixed user session features...
Train shape after user session features: (770164, 444)
Test shape after user session features: (369301, 443)


In [9]:
# Cell 8: Fixed Page Context Features
def create_page_context_features_fixed(df):
    """Create page context features with proper temporal ordering"""
    df_features = df.copy()
    
    # Page type encoding
    df_features['page_type'] = df_features['id6']
    
    # One-hot encode page types
    page_dummies = pd.get_dummies(df_features['page_type'], prefix='page')
    df_features = pd.concat([df_features, page_dummies], axis=1)
    
    # FIXED: Page performance with expanding window (no future data)
    df_features = df_features.sort_values(['id6', 'impression_time']).reset_index(drop=True)
    
    # Expanding page statistics (only uses historical data)
    df_features['page_cumulative_impressions'] = df_features.groupby('id6').cumcount() + 1
    df_features['page_cumulative_clicks'] = df_features.groupby('id6')['has_clicked'].cumsum()
    
    # Page CTR using only historical data
    df_features['page_historical_ctr'] = (
        df_features['page_cumulative_clicks'] / df_features['page_cumulative_impressions']
    )
    
    # Page engagement level based on historical performance
    df_features['page_engagement_tier'] = pd.cut(
        df_features['page_historical_ctr'],
        bins=[0, 0.02, 0.05, 0.1, 1.0],
        labels=[1, 2, 3, 4]
    ).fillna(1).astype(int)
    
    # FIXED: User-page interaction patterns (expanding window)
    df_features = df_features.sort_values(['id2', 'id6', 'impression_time']).reset_index(drop=True)
    
    df_features['user_page_cumulative_impressions'] = df_features.groupby(['id2', 'id6']).cumcount() + 1
    df_features['user_page_cumulative_clicks'] = df_features.groupby(['id2', 'id6'])['has_clicked'].cumsum()
    
    # User-page historical CTR
    df_features['user_page_historical_ctr'] = (
        df_features['user_page_cumulative_clicks'] / df_features['user_page_cumulative_impressions']
    )
    
    # Page preference score (user's historical performance vs page average)
    df_features['user_page_preference'] = (
        df_features['user_page_historical_ctr'] / (df_features['page_historical_ctr'] + 0.001)
    )
    
    return df_features

# Apply fixed page context features
print("Creating fixed page context features...")
train_df_features = create_page_context_features_fixed(train_df_features)
test_df_features = create_page_context_features_fixed(test_df_features)
print(f"Train shape after page context features: {train_df_features.shape}")
print(f"Test shape after page context features: {test_df_features.shape}")


Creating fixed page context features...
Train shape after page context features: (770164, 453)
Test shape after page context features: (369301, 452)


## Phase 2 - Advanced Features

In [10]:
# Cell 9: Enhanced Interest-Offer Alignment Features
def create_enhanced_interest_offer_alignment(df):
    """Enhanced interest-offer alignment with gap analysis and interactions"""
    df_features = df.copy()
    
    # Define interest categories mapping
    interest_mapping = {
        'automotive': ['f1'],
        'dining': ['f2', 'f8'],
        'business': ['f3', 'f5', 'f6', 'f10'],
        'hobby': ['f4', 'f12'],
        'home': ['f7'],
        'travel': ['f9'],
        'electronics': ['f11']
    }
    
    # Calculate category interest scores
    for category, features in interest_mapping.items():
        available_features = [f for f in features if f in df_features.columns]
        if available_features:
            df_features[f'interest_{category}_score'] = df_features[available_features].mean(axis=1)
        else:
            df_features[f'interest_{category}_score'] = 0
    
    # Overall interest metrics
    interest_cols = [f'f{i}' for i in range(1, 13) if f'f{i}' in df_features.columns]
    if interest_cols:
        df_features['interest_total_score'] = df_features[interest_cols].sum(axis=1)
        df_features['interest_diversity_score'] = (df_features[interest_cols] > 0).sum(axis=1)
        df_features['interest_strength_ratio'] = df_features[interest_cols].max(axis=1) / (df_features['interest_total_score'] + 1e-8)
        
        # Top interest identification
        df_features['top_interest_value'] = df_features[interest_cols].max(axis=1)
        df_features['top_interest_category'] = df_features[interest_cols].idxmax(axis=1)
        
        # Interest gap features (user vs category median)
        if 'f374' in df_features.columns:
            for i in range(1, 13):
                col = f'f{i}'
                if col in df_features.columns:
                    category_median = df_features.groupby('f374')[col].transform('median')
                    df_features[f'interest_gap_f{i}'] = df_features[col] - category_median
        
        # Cross-feature interactions
        key_interest_cols = ['f1', 'f2', 'f9', 'f11'] # automotive, dining, travel, electronics
        available_key_interests = [col for col in key_interest_cols if col in df_features.columns]
        
        if available_key_interests and 'f376' in df_features.columns:
            df_features['avg_key_interest'] = df_features[available_key_interests].mean(axis=1)
            df_features['discount_interest_ratio'] = df_features['f376'] / (df_features['avg_key_interest'] + 1e-3)
    
    return df_features

# Apply enhanced interest-offer alignment
print("Creating enhanced interest-offer alignment features...")
train_df_features = create_enhanced_interest_offer_alignment(train_df_features)
test_df_features = create_enhanced_interest_offer_alignment(test_df_features)
print(f"Train shape after interest features: {train_df_features.shape}")
print(f"Test shape after interest features: {test_df_features.shape}")


Creating enhanced interest-offer alignment features...
Train shape after interest features: (770164, 479)
Test shape after interest features: (369301, 478)


In [11]:
# Cell 10: Enhanced Loyalty Features
def create_enhanced_loyalty_features(df):
    """Enhanced loyalty features with cross-feature interactions"""
    df_features = df.copy()
    
    # Membership level numerical encoding
    membership_hierarchy = {
        'Basic': 1, 'Silver': 2, 'Gold': 3, 'Platinum': 4, 'Diamond': 5
    }
    
    if 'f42' in df_features.columns:
        df_features['loyalty_tier_numerical'] = df_features['f42'].map(membership_hierarchy).fillna(0)
        df_features['has_membership'] = (df_features['f42'].notna()).astype(int)
    else:
        df_features['loyalty_tier_numerical'] = 0
        df_features['has_membership'] = 0
    
    # Miles and points features
    if 'f43' in df_features.columns:
        df_features['f43'] = pd.to_numeric(df_features['f43'], errors='coerce')
        df_features['miles_balance_log'] = np.log1p(df_features['f43'].fillna(0))
        df_features['has_miles_balance'] = (df_features['f43'] > 0).astype(int)
    
    if 'f47' in df_features.columns:
        df_features['f47'] = pd.to_numeric(df_features['f47'], errors='coerce')
        df_features['award_miles_log'] = np.log1p(df_features['f47'].fillna(0))
        df_features['has_award_miles'] = (df_features['f47'] > 0).astype(int)
    
    # Miles utilization ratio
    if 'f43' in df_features.columns and 'f47' in df_features.columns:
        df_features['miles_utilization_ratio'] = df_features['f47'] / (df_features['f43'] + 1e-8)
    
    # Elite status features
    if 'f46' in df_features.columns:
        df_features['f46'] = pd.to_numeric(df_features['f46'], errors='coerce')
        df_features['elite_qualifying_miles_log'] = np.log1p(df_features['f46'].fillna(0))
        df_features['has_elite_activity'] = (df_features['f46'] > 0).astype(int)
        df_features['elite_status_binary'] = (df_features['f46'] >= 25000).astype(int)
        
        # Elite tier numerical
        elite_tier_temp = pd.cut(
            df_features['f46'],
            bins=[0, 25000, 50000, 75000, float('inf')],
            labels=[0, 1, 2, 3]
        )
        df_features['elite_tier_numerical'] = elite_tier_temp.fillna(0).astype(int)
    
    # Customer value tier
    if 'f53' in df_features.columns:
        df_features['f53'] = pd.to_numeric(df_features['f53'], errors='coerce')
        df_features['customer_value_tier'] = df_features['f53'].fillna(1)
        df_features['is_high_value_customer'] = (df_features['f53'] >= 4).astype(int)
        
        # Value tier discount interaction
        if 'f376' in df_features.columns:
            discount_mean = df_features['f376'].mean()
            discount_std = df_features['f376'].std()
            df_features['discount_z_score'] = (df_features['f376'] - discount_mean) / (discount_std + 1e-8)
            df_features['value_tier_discount'] = df_features['customer_value_tier'] * df_features['discount_z_score']
    
    # Loyalty program tenure
    if 'f58' in df_features.columns:
        df_features['f58'] = pd.to_numeric(df_features['f58'], errors='coerce')
        df_features['loyalty_tenure_years'] = df_features['f58'] / 365.25
        df_features['is_new_member'] = (df_features['f58'] <= 365).astype(int)
        
        # Tenure category as numerical
        tenure_bins = pd.cut(df_features['f58'], bins=[0, 365, 1095, 1825, float('inf')], labels=[1, 2, 3, 4])
        df_features['loyalty_tenure_category_num'] = tenure_bins.fillna(1).astype(int)
    
    # Composite loyalty engagement score
    loyalty_components = []
    for comp in ['loyalty_tier_numerical', 'has_miles_balance', 'has_elite_activity', 'customer_value_tier']:
        if comp in df_features.columns:
            loyalty_components.append(comp)
    
    if loyalty_components:
        # Normalize components
        normalized_components = []
        for component in loyalty_components:
            col_min = df_features[component].min()
            col_max = df_features[component].max()
            if col_max > col_min:
                normalized = (df_features[component] - col_min) / (col_max - col_min)
                normalized_components.append(normalized)
        
        if normalized_components:
            df_features['loyalty_engagement_score'] = pd.concat(normalized_components, axis=1).mean(axis=1)
    
    return df_features

# Apply enhanced loyalty features
print("Creating enhanced loyalty features...")
train_df_features = create_enhanced_loyalty_features(train_df_features)
test_df_features = create_enhanced_loyalty_features(test_df_features)
print(f"Train shape after loyalty features: {train_df_features.shape}")
print(f"Test shape after loyalty features: {test_df_features.shape}")


Creating enhanced loyalty features...
Train shape after loyalty features: (770164, 498)
Test shape after loyalty features: (369301, 497)


In [12]:
# Cell 11: Enhanced Offer Competition Features
def create_offer_competition_features(df):
    """Create offer competition and context features"""
    df_features = df.copy()
    
    # Ensure we have required columns
    if 'f374' not in df_features.columns or 'impression_time' not in df_features.columns:
        print("Warning: Required columns missing for offer competition features")
        return df_features
    
    # Daily offer competition in same category
    df_features['offers_in_same_category'] = df_features.groupby([
        'id2', 'f374', 
        pd.Grouper(key='impression_time', freq='D')
    ])['id3'].transform('count')
    
    # Offer density features
    df_features['daily_offer_density'] = df_features.groupby([
        'id2', 
        pd.Grouper(key='impression_time', freq='D')
    ])['id3'].transform('count')
    
    # Category competition level
    df_features['category_competition_level'] = pd.cut(
        df_features['offers_in_same_category'],
        bins=[0, 1, 3, 5, float('inf')],
        labels=[1, 2, 3, 4]
    ).fillna(1).astype(int)
    
    # Offer uniqueness score
    df_features['offer_uniqueness'] = 1 / (df_features['offers_in_same_category'] + 1)
    
    # Time-based offer frequency
    df_features['hourly_offer_frequency'] = df_features.groupby([
        'id2', 
        pd.Grouper(key='impression_time', freq='H')
    ])['id3'].transform('count')
    
    return df_features

# Apply offer competition features
print("Creating offer competition features...")
train_df_features = create_offer_competition_features(train_df_features)
test_df_features = create_offer_competition_features(test_df_features)
print(f"Train shape after competition features: {train_df_features.shape}")
print(f"Test shape after competition features: {test_df_features.shape}")


Creating offer competition features...
Train shape after competition features: (770164, 503)
Test shape after competition features: (369301, 502)


## Phase 3 - Time-Based Data Splitting and Feature Selection

In [13]:
# Cell 12: FIXED Time-Based Data Splitting Function
def create_time_based_split(df, time_threshold=None, train_ratio=0.8, method='quantile'):
    """Create time-based train/validation split to prevent data leakage"""
    
    if 'impression_time' not in df.columns:
        print("Warning: impression_time column not found")
        return df, None
    
    # Calculate threshold dynamically if not provided
    if time_threshold is None:
        if method == 'quantile':
            time_threshold = df['impression_time'].quantile(train_ratio)
        elif method == 'sample_based':
            sorted_dates = df['impression_time'].sort_values()
            split_index = int(len(sorted_dates) * train_ratio)
            time_threshold = sorted_dates.iloc[split_index]
        elif method == 'daily_boundary':
            daily_counts = df.groupby(df['impression_time'].dt.date).size()
            daily_counts_cumsum = daily_counts.cumsum()
            target_train_samples = int(len(df) * train_ratio)
            split_day = daily_counts_cumsum[daily_counts_cumsum >= target_train_samples].index[0]
            time_threshold = pd.Timestamp(split_day)
        else:
            # Default to quantile method
            time_threshold = df['impression_time'].quantile(train_ratio)
        
        print(f"Auto-calculated threshold using {method} method: {time_threshold}")
    else:
        time_threshold = pd.Timestamp(time_threshold)
        print(f"Using provided threshold: {time_threshold}")
    
    # Split data
    train_temporal = df[df['impression_time'] < time_threshold].copy()
    val_temporal = df[df['impression_time'] >= time_threshold].copy()
    
    # Validation check
    if len(val_temporal) > 0:
        # Ensure no data leakage
        if len(train_temporal) > 0:
            assert val_temporal['impression_time'].min() >= train_temporal['impression_time'].max(), \
                "Data leakage detected: validation data overlaps with training data"
        
        print(f"Time-based split created successfully:")
        print(f" Training: {len(train_temporal)} samples ({len(train_temporal)/len(df):.1%})")
        print(f" Validation: {len(val_temporal)} samples ({len(val_temporal)/len(df):.1%})")
        print(f" Training time range: {train_temporal['impression_time'].min()} to {train_temporal['impression_time'].max()}")
        print(f" Validation time range: {val_temporal['impression_time'].min()} to {val_temporal['impression_time'].max()}")
        
        return train_temporal, val_temporal
    else:
        print("Warning: No validation data found after time threshold")
        print(f"Threshold: {time_threshold}")
        print(f"Data range: {df['impression_time'].min()} to {df['impression_time'].max()}")
        return train_temporal, None

# Test the function with your data
print("Testing time-based split function...")
test_train, test_val = create_time_based_split(train_df_features, method='quantile', train_ratio=0.8)


Testing time-based split function...
Auto-calculated threshold using quantile method: 2023-11-03 08:30:56.586400
Time-based split created successfully:
 Training: 616131 samples (80.0%)
 Validation: 154033 samples (20.0%)
 Training time range: 2023-11-01 00:00:23.729000 to 2023-11-03 08:30:56.584000
 Validation time range: 2023-11-03 08:30:56.590000 to 2023-11-03 23:59:58.572000


In [14]:
# Cell 13: Feature Importance-Based Selection
def feature_importance_selection(train_df, target_col='y', n_features=400):
    """Select features based on LightGBM feature importance"""
    
    if target_col not in train_df.columns:
        print(f"Warning: Target column '{target_col}' not found")
        return train_df.columns.tolist()
    
    # Prepare features and target
    feature_cols = [col for col in train_df.columns 
                   if not col.startswith('id') and col != target_col 
                   and col not in ['impression_time', 'click_time']]
    
    X = train_df[feature_cols].fillna(0) # Fill NaN for LightGBM
    y = train_df[target_col]
    
    print(f"Training LightGBM for feature selection on {len(feature_cols)} features...")
    
    try:
        # Train LightGBM model
        model = LGBMClassifier(
            n_estimators=100,
            random_state=42,
            verbose=-1
        )
        model.fit(X, y)
        
        # Get feature importance
        importance = pd.Series(model.feature_importances_, index=feature_cols)
        
        # Select top N features
        top_features = importance.nlargest(n_features).index.tolist()
        
        print(f"Selected top {len(top_features)} features based on importance")
        print(f"Top 10 most important features:")
        for i, (feat, imp) in enumerate(importance.nlargest(10).items()):
            print(f" {i+1}. {feat}: {imp:.4f}")
        
        # Add back ID columns and target
        id_cols = [col for col in train_df.columns if col.startswith('id')]
        final_features = id_cols + top_features + [target_col]
        
        return final_features
    
    except Exception as e:
        print(f"Error in feature selection: {e}")
        return train_df.columns.tolist()

print("Feature importance selection function defined")


Feature importance selection function defined


## Phase 4 - Final Processing and Cleanup

In [15]:
# Cell 14: Comprehensive Feature Cleanup
def comprehensive_feature_cleanup(train_df, test_df):
    """Comprehensive cleanup of engineered features"""
    
    # Remove datetime columns that cause issues
    datetime_columns_to_remove = [
        'time_of_day'
    ]
    
    # Remove categorical columns (keep numerical encodings)
    categorical_columns_to_remove = [
        'f374_top', 'f374', 'page_type', 'top_interest_category'
    ]
    
    # Combine all columns to remove
    columns_to_remove = datetime_columns_to_remove + categorical_columns_to_remove
    
    # Clean train dataset
    train_cleaned = train_df.copy()
    for col in columns_to_remove:
        if col in train_cleaned.columns:
            train_cleaned = train_cleaned.drop(columns=[col])
    
    # Clean test dataset
    test_cleaned = test_df.copy()
    for col in columns_to_remove:
        if col in test_cleaned.columns:
            test_cleaned = test_cleaned.drop(columns=[col])
    
    # Ensure column consistency
    train_cols = set(train_cleaned.columns)
    test_cols = set(test_cleaned.columns)
    
    common_cols = train_cols.intersection(test_cols)
    train_only_cols = train_cols - test_cols
    test_only_cols = test_cols - train_cols
    
    # Add missing columns with zeros
    for col in train_only_cols:
        if col != 'y': # Don't add target to test
            test_cleaned[col] = 0
    
    for col in test_only_cols:
        train_cleaned[col] = 0
    
    print(f"Cleanup completed:")
    print(f" Train shape: {train_cleaned.shape}")
    print(f" Test shape: {test_cleaned.shape}")
    print(f" Columns removed: {len(columns_to_remove)}")
    
    return train_cleaned, test_cleaned

# Apply comprehensive cleanup
print("Applying comprehensive feature cleanup...")
train_df_final, test_df_final = comprehensive_feature_cleanup(train_df_features, test_df_features)
print("Cleanup completed successfully")


Applying comprehensive feature cleanup...
Cleanup completed:
 Train shape: (770164, 498)
 Test shape: (369301, 497)
 Columns removed: 5
Cleanup completed successfully


In [16]:
# # Cell 15: FIXED Apply Feature Selection and Final Processing

# def finalize_feature_engineering(train_df, test_df):
#     """Apply feature selection and final processing with proper categorical handling and temporal column preservation"""
    
#     # STEP 1: Handle categorical columns before feature selection
#     train_processed = train_df.copy()
#     test_processed = test_df.copy()
    
#     # Identify categorical columns (object dtype)
#     categorical_cols = train_processed.select_dtypes(include=['object']).columns.tolist()
    
#     # Remove target column from categorical processing if present
#     if 'y' in categorical_cols:
#         categorical_cols.remove('y')
    
#     # Remove temporal columns from categorical processing
#     temporal_cols = ['impression_time', 'click_time']
#     for col in temporal_cols:
#         if col in categorical_cols:
#             categorical_cols.remove(col)
    
#     # **CRITICAL FIX: Remove ID columns from categorical processing**
#     id_columns = [col for col in categorical_cols if col.startswith('id')]
#     for col in id_columns:
#         if col in categorical_cols:
#             categorical_cols.remove(col)
    
#     print(f"Found {len(categorical_cols)} categorical columns to process: {categorical_cols}")
#     print(f"Excluding ID columns from categorical processing: {id_columns}")
    
#     # Process categorical columns (now excluding ID columns)
#     for col in categorical_cols:
#         if col in train_processed.columns:
#             # Method 1: Label Encoding for ordinal categories
#             if col in ['f42', 'f54', 'f55']: # Membership/status columns
#                 # Create combined categories for consistent encoding
#                 combined_values = pd.concat([
#                     train_processed[col].dropna(), 
#                     test_processed[col].dropna() if col in test_processed.columns else pd.Series()
#                 ]).unique()
                
#                 # Create label encoder
#                 le = LabelEncoder()
#                 le.fit(combined_values)
                
#                 # Apply encoding
#                 train_processed[col] = train_processed[col].fillna('Unknown')
#                 train_processed[col] = le.transform(train_processed[col])
                
#                 if col in test_processed.columns:
#                     test_processed[col] = test_processed[col].fillna('Unknown')
#                     # Handle unseen categories
#                     test_processed[col] = test_processed[col].apply(
#                         lambda x: le.transform([x])[0] if x in le.classes_ else -1
#                     )
#             # Method 2: Frequency encoding for high-cardinality categories
#             else:
#                 # Calculate frequency encoding from training data
#                 freq_encoding = train_processed[col].value_counts().to_dict()
                
#                 # Apply frequency encoding
#                 train_processed[col] = train_processed[col].map(freq_encoding).fillna(0)
                
#                 if col in test_processed.columns:
#                     test_processed[col] = test_processed[col].map(freq_encoding).fillna(0)
    
#     # STEP 2: Clean feature names for LightGBM compatibility
#     def clean_feature_names_for_lightgbm(df):
#         """Clean column names to remove special JSON characters"""
#         import re
#         new_names = {}
#         seen = set()
        
#         for col in df.columns:
#             # Remove all characters except letters, numbers, and underscore
#             clean_col = re.sub(r'[^A-Za-z0-9_]+', '', col)
            
#             # Handle empty names
#             if not clean_col:
#                 clean_col = 'feature'
            
#             # Ensure unique column names
#             if clean_col in seen:
#                 i = 1
#                 new_col = f"{clean_col}_{i}"
#                 while new_col in seen:
#                     i += 1
#                     new_col = f"{clean_col}_{i}"
#                 clean_col = new_col
            
#             seen.add(clean_col)
#             new_names[col] = clean_col
        
#         return df.rename(columns=new_names)
    
#     # Apply feature name cleaning
#     print("Cleaning feature names for LightGBM compatibility...")
#     train_processed = clean_feature_names_for_lightgbm(train_processed)
#     test_processed = clean_feature_names_for_lightgbm(test_processed)
    
#     # STEP 3: Preserve temporal columns before feature selection
#     temporal_cols = ['impression_time', 'click_time']
#     preserved_temporal = {}
    
#     for col in temporal_cols:
#         if col in train_processed.columns:
#             preserved_temporal[f'train_{col}'] = train_processed[col].copy()
#         if col in test_processed.columns:
#             preserved_temporal[f'test_{col}'] = test_processed[col].copy()
    
#     # STEP 4: Prepare features for selection (exclude temporal columns and ID columns)
#     feature_cols = [col for col in train_processed.columns 
#                     if not col.startswith('id') and col != 'y' 
#                     and col not in temporal_cols]
    
#     # Convert any remaining non-numeric columns
#     for col in feature_cols:
#         if col in train_processed.columns:
#             train_processed[col] = pd.to_numeric(train_processed[col], errors='coerce').fillna(0)
#         if col in test_processed.columns:
#             test_processed[col] = pd.to_numeric(test_processed[col], errors='coerce').fillna(0)
    
#     print(f"All categorical columns processed and feature names cleaned. Ready for feature selection.")
    
#     # STEP 5: Apply feature importance selection (excluding temporal columns)
#     selected_features = feature_importance_selection(train_processed, target_col='y', n_features=400)
    
#     # STEP 6: Add back temporal columns to final selection
#     final_selected_features = selected_features.copy()
    
#     # Add temporal columns back if they exist
#     for col in temporal_cols:
#         if col in train_processed.columns and col not in final_selected_features:
#             final_selected_features.append(col)
    
#     print(f"Preserved temporal columns: {[col for col in temporal_cols if col in train_processed.columns]}")
    
#     # Apply selection to both datasets
#     train_selected = train_processed[final_selected_features].copy()
#     test_selected_features = [col for col in final_selected_features if col in test_processed.columns]
#     test_selected = test_processed[test_selected_features].copy()
    
#     # STEP 7: Final imputation for critical features
#     critical_features = [
#         'user_rolling_ctr_7d', 'user_rolling_ctr_3d', 'page_historical_ctr',
#         'user_page_historical_ctr', 'interest_total_score', 'loyalty_engagement_score'
#     ]
    
#     for feature in critical_features:
#         if feature in train_selected.columns:
#             train_selected[feature] = train_selected[feature].fillna(0)
#         if feature in test_selected.columns:
#             test_selected[feature] = test_selected[feature].fillna(0)
    
#     print(f"Final feature engineering completed:")
#     print(f" Final train shape: {train_selected.shape}")
#     print(f" Final test shape: {test_selected.shape}")
#     print(f" Features selected (excluding temporal): {len([f for f in final_selected_features if f not in temporal_cols and f != 'y'])}")
#     print(f" Temporal columns preserved: {[col for col in temporal_cols if col in train_selected.columns]}")
    
#     return train_selected, test_selected


# # Apply final feature engineering
# print("Applying final feature engineering...")
# train_final, test_final = finalize_feature_engineering(train_df_final, test_df_final)
# print("Final processing completed")


In [17]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
# import gc

# def finalize_feature_engineering(train_df, test_df):
#     """Optimized: Feature engineering with minimal RAM and no data quality loss"""

#     # --- STEP 1: Identify column roles early ---
#     temporal_cols = ['impression_time', 'click_time']
#     target_col = 'y'
#     id_cols = [c for c in train_df.columns if c.startswith('id')]
    
#     # Categoricals (excluding target, temporal, id columns)
#     categorical_cols = [
#         c for c in train_df.select_dtypes(include='object').columns 
#         if c not in temporal_cols + [target_col] and not c.startswith('id')
#     ]
#     print(f"Categoricals: {categorical_cols}")

#     # --- STEP 2: Column selection (early filtering) ---
#     feature_cols = [
#         c for c in train_df.columns 
#         if c not in id_cols + temporal_cols + [target_col]
#     ]
#     # Add back temporal for final output after selection

#     # --- STEP 3: Downcast numerics, set categoricals dtype ---
#     for col in feature_cols:
#         if train_df[col].dtype in ['int64', 'float64']:
#             train_df[col] = pd.to_numeric(train_df[col], downcast='float')
#             if col in test_df:
#                 test_df[col] = pd.to_numeric(test_df[col], downcast='float')
#     for col in categorical_cols:
#         train_df[col] = train_df[col].astype('category')
#         if col in test_df.columns:
#             test_df[col] = test_df[col].astype('category')

#     gc.collect()

#     # --- STEP 4: Efficient encoding for categoricals ---
#     for col in categorical_cols:
#         if col in ['f42', 'f54', 'f55']:   # likely ordinal
#             le = LabelEncoder()
#             # Combine categories found in train and test for symmetry
#             all_vals = pd.concat([
#                 train_df[col], test_df[col] if col in test_df else pd.Series([], dtype="category")
#             ]).astype(str)
#             le.fit(all_vals)
#             train_df[col] = le.transform(train_df[col].astype(str))
#             if col in test_df:
#                 # Map test values; unknown to -1
#                 valid_map = {cat: idx for idx, cat in enumerate(le.classes_)}
#                 test_df[col] = test_df[col].astype(str).map(valid_map).fillna(-1).astype(int)
#         else:
#             # Frequency encode (fast, vectorized)
#             freqs = train_df[col].value_counts(dropna=False)
#             train_df[col] = train_df[col].map(freqs).astype(np.float32)
#             if col in test_df:
#                 test_df[col] = test_df[col].map(freqs).fillna(0).astype(np.float32)
#     gc.collect()

#     # --- STEP 5: Clean column names for LightGBM ---
#     import re
#     def clean_names(df):
#         renamer = {}
#         existing = set()
#         for c in df.columns:
#             name = re.sub(r'[^A-Za-z0-9_]+', '', c)
#             if not name: name = 'feature'
#             if name in existing:
#                 base = name
#                 idx = 1
#                 while f"{base}_{idx}" in existing: idx += 1
#                 name = f"{base}_{idx}"
#             existing.add(name)
#             renamer[c] = name
#         return df.rename(columns=renamer)
#     train_df = clean_names(train_df)
#     test_df  = clean_names(test_df)

#     # --- STEP 6: Feature selection (excluding temporals & id) ---
#     # Only pass necessary columns & target to selector
#     train_features = [c for c in train_df.columns if c not in id_cols + temporal_cols + [target_col]]
#     # <--- CALL YOUR SMART SELECTION FUNCTION (must exist in scope) --->
#     selected_features = feature_importance_selection(
#         train_df[train_features + [target_col]],
#         target_col=target_col, 
#         n_features=400
#     )
#     # Add back temporal columns if present
#     for col in temporal_cols:
#         if col in train_df.columns and col not in selected_features:
#             selected_features.append(col)

#     print(f"Selected features: {len(selected_features)}")

#     # --- STEP 7: Assemble & impute final outputs ---
#     train_final = train_df[selected_features + [target_col]].copy()
#     test_final  = test_df[[c for c in selected_features if c in test_df]].copy()

#     # Final imputation for critical features
#     critical_features = [
#         'user_rolling_ctr_7d', 'user_rolling_ctr_3d', 'page_historical_ctr',
#         'user_page_historical_ctr', 'interest_total_score', 'loyalty_engagement_score'
#     ]
#     for col in critical_features:
#         if col in train_final:
#             train_final[col] = train_final[col].fillna(0)
#         if col in test_final:
#             test_final[col] = test_final[col].fillna(0)
#     gc.collect()

#     print(f"Final train shape: {train_final.shape}")
#     print(f"Final test shape:  {test_final.shape}")
#     print(f"Preserved temporals: {[c for c in temporal_cols if c in train_final]}")
#     return train_final, test_final

# # Run finalized feature engineering
# print("Applying optimized, memory-efficient feature engineering...")
# train_final, test_final = finalize_feature_engineering(train_df_final, test_df_final)
# print("Done.")


In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import gc
import re

def finalize_feature_engineering(train_df, test_df):
    """Optimized, safe feature engineering: memory friendly, with IDs preserved for validation/export."""

    # ----------- IDENTIFY ROLES ---------------
    target_col = 'y'
    temporal_cols = ['impression_time', 'click_time']
    id_cols = [c for c in train_df.columns if c.startswith('id')]
    
    # Categorical (non-ID, non-temporal, non-target)
    categorical_cols = [
        c for c in train_df.select_dtypes(include='object').columns 
        if c not in id_cols + temporal_cols + [target_col]
    ]
    print(f"[FE] Categorical columns: {categorical_cols}")
    
    # --------- EARLY COLUMN FILTERING ----------
    feature_cols = [
        c for c in train_df.columns 
        if c not in id_cols + temporal_cols + [target_col]
    ]

    # ------------ DOWNSCALING TYPES --------------
    for col in feature_cols:
        if train_df[col].dtype in ('int64', 'float64'):
            train_df[col] = pd.to_numeric(train_df[col], downcast='float')
            if col in test_df:
                test_df[col] = pd.to_numeric(test_df[col], downcast='float')
    for col in categorical_cols:
        # Use pandas categorical to save immense RAM
        train_df[col] = train_df[col].astype('category')
        if col in test_df.columns:
            test_df[col] = test_df[col].astype('category')
    gc.collect()

    # ------------ ENCODING CATEGORICALS -------------
    for col in categorical_cols:
        if col in ['f42', 'f54', 'f55']:
            # Label encode with classes from both train and test
            le = LabelEncoder()
            all_vals = pd.concat([
                train_df[col], test_df[col] if col in test_df.columns else pd.Series([], dtype="category")
            ]).astype(str)
            le.fit(all_vals)
            train_df[col] = le.transform(train_df[col].astype(str))
            if col in test_df:
                valid_map = {cat: idx for idx, cat in enumerate(le.classes_)}
                test_df[col] = test_df[col].astype(str).map(valid_map).fillna(-1).astype(int)
        else:
            # Frequency encode, vectorized
            freqs = train_df[col].value_counts(dropna=False)
            train_df[col] = train_df[col].map(freqs).astype(np.float32)
            if col in test_df.columns:
                test_df[col] = test_df[col].map(freqs).fillna(0).astype(np.float32)
    gc.collect()

    # ------------ LIGHTGBM-FRIENDLY COLUMN NAMES ------------
    def clean_names(df):
        renamer = {}
        existing = set()
        for c in df.columns:
            name = re.sub(r'[^A-Za-z0-9_]+', '', c)
            if not name: name = 'feature'
            if name in existing:
                base = name
                idx = 1
                while f"{base}_{idx}" in existing: idx += 1
                name = f"{base}_{idx}"
            existing.add(name)
            renamer[c] = name
        return df.rename(columns=renamer)
    train_df = clean_names(train_df)
    test_df  = clean_names(test_df)
    gc.collect()

    # ------------ FEATURE SELECTION -----------------
    train_features = [
        c for c in train_df.columns
        if c not in id_cols + temporal_cols + [target_col]
    ]
    # You must provide your feature_importance_selection
    selected_features = feature_importance_selection(
        train_df[train_features + [target_col]],
        target_col=target_col,
        n_features=400
    )
    # Add back temporals for final DataFrame
    for col in temporal_cols:
        if col in train_df.columns and col not in selected_features:
            selected_features.append(col)

    # ------------ FINAL DF ASSEMBLY ----------------
    # Use .copy() ONCE to avoid SettingWithCopy/side-effects
    train_final = train_df[selected_features + [target_col]].copy()
    test_final  = test_df[[c for c in selected_features if c in test_df.columns]].copy()

    # ------------ CRITICAL IMPUTATION --------------
    critical_features = [
        'user_rolling_ctr_7d', 'user_rolling_ctr_3d', 'page_historical_ctr',
        'user_page_historical_ctr', 'interest_total_score', 'loyalty_engagement_score'
    ]
    for col in critical_features:
        if col in train_final:
            train_final[col] = train_final[col].fillna(0)
        if col in test_final:
            test_final[col] = test_final[col].fillna(0)
    gc.collect()

    # ------------ PRESERVE ID COLUMNS (validation/output only!) ----------------
    # (Preserve original order, don't use these as features for model input)
    for col in id_cols:
        if col not in train_final and col in train_df:
            train_final[col] = train_df[col].values
        if col not in test_final and col in test_df:
            test_final[col] = test_df[col].values

    print(f"[FE] Final train shape: {train_final.shape}")
    print(f"[FE] Final test shape:  {test_final.shape}")
    print(f"[FE] Preserved temporals: {[c for c in temporal_cols if c in train_final]}  |  IDs: {id_cols}")
    return train_final, test_final

# For usage:
print("Applying optimized feature engineering (with ID columns preserved for validation/export only)...")
train_final, test_final = finalize_feature_engineering(train_df_final, test_df_final)
print("Done.")



Applying optimized feature engineering (with ID columns preserved for validation/export only)...
[FE] Categorical columns: ['f42', 'f50', 'f52', 'f54', 'f55', 'f56', 'f57', 'f354', 'f378']
Training LightGBM for feature selection on 484 features...
Selected top 400 features based on importance
Top 10 most important features:
 1. time_since_last_impression: 550.0000
 2. user_session_length: 85.0000
 3. f366: 68.0000
 4. hourly_offer_frequency: 58.0000
 5. f223: 47.0000
 6. f132: 46.0000
 7. f207: 46.0000
 8. user_cumulative_impressions: 46.0000
 9. f363: 45.0000
 10. user_avg_session_length: 43.0000
[FE] Final train shape: (770164, 415)
[FE] Final test shape:  (369301, 413)
[FE] Preserved temporals: ['impression_time', 'click_time']  |  IDs: ['id1', 'id2', 'id3', 'id4', 'id5', 'id10', 'id8', 'id12', 'id13', 'id6', 'id7']
Done.


In [19]:
# New Cell: Data Integrity Validation
def validate_data_integrity(train_df, test_df):
    """Validate that ID columns and datetime columns haven't been corrupted"""
    
    print("=== DATA INTEGRITY VALIDATION ===")
    
    # Check ID columns
    id_columns = [col for col in train_df.columns if col.startswith('id')]
    print(f"ID columns found: {id_columns}")
    
    for col in id_columns:
        if col in train_df.columns:
            # Check if ID values look reasonable (not frequency encoded)
            unique_count = train_df[col].nunique()
            total_count = len(train_df)
            uniqueness_ratio = unique_count / total_count
            
            print(f"  {col}: {unique_count} unique values, ratio: {uniqueness_ratio:.4f}")
            
            # Flag if ID column looks corrupted (too low uniqueness for IDs)
            if col in ['id1', 'id2', 'id3'] and uniqueness_ratio < 0.1:
                print(f"  WARNING: {col} may be corrupted (low uniqueness ratio)")
    
    # Check datetime columns
    datetime_cols = ['impression_time', 'click_time']
    for col in datetime_cols:
        if col in train_df.columns:
            min_date = train_df[col].min()
            max_date = train_df[col].max()
            
            print(f"  {col}: {min_date} to {max_date}")
            
            # Flag if dates reset to 1970
            if pd.to_datetime(min_date).year < 2000:
                print(f"  ERROR: {col} contains dates before year 2000 - likely corrupted!")
            else:
                print(f"  ✓ {col} dates look valid")
    
    print("=== VALIDATION COMPLETE ===")

# Run validation
validate_data_integrity(train_final, test_final)


=== DATA INTEGRITY VALIDATION ===
ID columns found: ['id1', 'id2', 'id3', 'id4', 'id5', 'id10', 'id8', 'id12', 'id13', 'id6', 'id7']
  id1: 770164 unique values, ratio: 1.0000
  id2: 46550 unique values, ratio: 0.0604
  id3: 757 unique values, ratio: 0.0010
  id4: 763680 unique values, ratio: 0.9916
  id5: 3 unique values, ratio: 0.0000
  id10: 2 unique values, ratio: 0.0000
  id8: 186 unique values, ratio: 0.0002
  id12: 108 unique values, ratio: 0.0001
  id13: 119 unique values, ratio: 0.0002
  id6: 0 unique values, ratio: 0.0000
  id7: 0 unique values, ratio: 0.0000
  impression_time: 2023-11-01 00:00:23.729000 to 2023-11-03 23:59:58.572000
  ✓ impression_time dates look valid
  click_time: NaT to NaT
  ✓ click_time dates look valid
=== VALIDATION COMPLETE ===


In [20]:
# Cell 16: FIXED Save Final Datasets with Dynamic Threshold
def save_final_datasets(train_df, test_df, optimal_threshold=None):
    """Save the final processed datasets with proper time-based validation split"""
    
    import os
    os.makedirs('dataset/stage_final', exist_ok=True)
    
    # Save datasets
    train_df.to_csv('dataset/stage_final/train_features_final.csv', index=False)
    test_df.to_csv('dataset/stage_final/test_features_final.csv', index=False)
    
    # Save feature names
    feature_names = [col for col in train_df.columns if col != 'y']
    pd.DataFrame({'feature_name': feature_names}).to_csv('dataset/stage_final/final_feature_names.csv', index=False)
    
    print(f"Final datasets saved:")
    print(f" Train: dataset/stage_final/train_features_final.csv ({train_df.shape})")
    print(f" Test: dataset/stage_final/test_features_final.csv ({test_df.shape})")
    print(f" Features: dataset/stage_final/final_feature_names.csv ({len(feature_names)} features)")
    
    # FIXED: Use dynamic threshold calculation
    if optimal_threshold is None:
        # Calculate threshold based on your data
        optimal_threshold = train_df['impression_time'].quantile(0.8)
        print(f"Using auto-calculated threshold: {optimal_threshold}")
    else:
        print(f"Using provided threshold: {optimal_threshold}")
    
    # Create time-based validation split with dynamic threshold
    train_temporal, val_temporal = create_time_based_split(
        train_df, 
        time_threshold=optimal_threshold,
        method='quantile'
    )
    
    if val_temporal is not None and len(val_temporal) > 0:
        train_temporal.to_csv('dataset/stage_final/train_temporal.csv', index=False)
        val_temporal.to_csv('dataset/stage_final/val_temporal.csv', index=False)
        print(f" Temporal split saved:")
        print(f" Train temporal: dataset/stage_final/train_temporal.csv ({train_temporal.shape})")
        print(f" Val temporal: dataset/stage_final/val_temporal.csv ({val_temporal.shape})")
    else:
        print("Warning: Could not create validation split. Saving full training set only.")
        train_df.to_csv('dataset/stage_final/train_temporal.csv', index=False)

# Save final datasets using the optimal threshold calculated earlier
print("Saving final datasets...")
save_final_datasets(train_final, test_final, optimal_threshold=optimal_threshold)

print("\n=== FEATURE ENGINEERING PIPELINE COMPLETED ===")
print("✓ Data leakage issues fixed")
print("✓ Enhanced temporal features added")
print("✓ Improved offer-user affinity features")
print("✓ Feature selection applied")
print("✓ Time-based validation split created with DYNAMIC threshold")
print("✓ All datasets saved successfully")


Saving final datasets...
Final datasets saved:
 Train: dataset/stage_final/train_features_final.csv ((770164, 415))
 Test: dataset/stage_final/test_features_final.csv ((369301, 413))
 Features: dataset/stage_final/final_feature_names.csv (413 features)
Using provided threshold: 2023-11-03 08:30:56.586400
Using provided threshold: 2023-11-03 08:30:56.586400
Time-based split created successfully:
 Training: 616131 samples (80.0%)
 Validation: 154033 samples (20.0%)
 Training time range: 2023-11-01 00:00:23.729000 to 2023-11-03 08:30:56.584000
 Validation time range: 2023-11-03 08:30:56.590000 to 2023-11-03 23:59:58.572000
 Temporal split saved:
 Train temporal: dataset/stage_final/train_temporal.csv ((616131, 415))
 Val temporal: dataset/stage_final/val_temporal.csv ((154033, 415))

=== FEATURE ENGINEERING PIPELINE COMPLETED ===
✓ Data leakage issues fixed
✓ Enhanced temporal features added
✓ Improved offer-user affinity features
✓ Feature selection applied
✓ Time-based validation split 

In [21]:
print(set(train_final.columns) - set(test_final.columns))


{'y'}


In [22]:
print(train_final.columns.sum())

AttributeError: 'Index' object has no attribute 'sum'

In [ ]:
from collections import Counter

# For any DataFrame, e.g., train_final
col_counts = Counter(train_final.columns)
# Only show columns that appear more than once
for col, freq in col_counts.items():
    if freq > 1:
        print(f"Column: {col}   Frequency: {freq}")



In [ ]:
train_final['y']

In [ ]:
train_df['y']

In [ ]:
print(train_final['y'].nunique())
